In [79]:
# Imports
from collections import defaultdict, Counter, deque
from functools import lru_cache, reduce
import itertools
import json
import numpy as np
import numpy.typing as npt
import re
from sortedcontainers import SortedList
from typing import Callable, TypeVar, Union, Optional


T = TypeVar('T')

# Helper functions

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

def split_csv_row(row: str) -> list[int]:
  return [int(x) for x in row.split(',')]

def split_int_row(row: str) -> list[int]:
  return [int(x) for x in row]

# Day 1

In [19]:
def parse_day1(data):
  elves = SortedList()
  current = 0
  for food in data:
    if not food:
      elves.add(current)
      current = 0
    else:
      current += int(food)
  return elves

data1 = parse_day1(data(1))

In [21]:
data1[-1]

71780

In [23]:
sum(data1[-3:])

212489

# Day 2

In [13]:
def parse2(data):
  result = Counter()
  for round in data:
    result[round] += 1
  return result
data2 = parse2(data(2))

In [11]:
score = {
  'A X': 3 + 1, # Rock Rock
  'A Y': 6 + 2, # Rock Paper
  'A Z': 0 + 3, # Rock Scissor
  'B X': 0 + 1, # Paper Rock
  'B Y': 3 + 2, # Paper Paper
  'B Z': 6 + 3, # Paper Scissor
  'C X': 6 + 1, # Scissor Rock
  'C Y': 0 + 2, # Scissor Paper
  'C Z': 3 + 3, # Scissor Scissor
}
sum([score[x]*data2[x] for x in data2])

13924

In [12]:
score = {
  'A X': 0 + 3, # Rock Lose Sciss
  'A Y': 3 + 1, # Rock Draw Rock
  'A Z': 6 + 2, # Rock Win Paper
  'B X': 0 + 1, # Paper Lose Rock
  'B Y': 3 + 2, # Paper Draw Paper
  'B Z': 6 + 3, # Paper Win Sciss
  'C X': 0 + 2, # Scissor Lose Paper
  'C Y': 3 + 3, # Scissor Draw Sciss
  'C Z': 6 + 1, # Scissor Win Rock
}
sum([score[x]*data2[x] for x in data2])

13448

# Day 3

In [29]:
data3 = data(3)

def item_priority(item):
  if ord(item) > 96:
    return ord(item) - 96
  return ord(item) - 38

In [31]:
def rucksack_items(rucksack):
  length = len(rucksack)//2
  return rucksack[:length], rucksack[length:]
  
def day3(data):
  sum = 0
  for x in data:
    a, b = rucksack_items(x)
    sum += item_priority(list(set(a).intersection(set(b)))[0])
  return sum

day3(data3)

7727

In [36]:
def day3_2(data):
  sum = 0
  for i in range(len(data)//3):
    a, b, c = data[i*3:i*3+3]
    common = set(a).intersection(set(b)).intersection(set(c))
    sum += item_priority(list(common)[0])
  return sum

day3_2(data3)

2609

# Day 4

In [43]:
data4 = data(4)

In [41]:
def has_envelop(a, b):
  return a[0] <= b[0] and a[1] >= b[1]

def day4(data, f):
  count = 0
  for i in data:
    a, b = [[int(x) for x in elf.split('-')] for elf in i.split(',')]
    count += f(a, b) or f(b, a)
  return count

day4(data4, has_envelop)

2

In [44]:
def has_overlap(a, b):
  return (a[0] <= b[0] and a[1] >= b[0]) or (a[0] >= b[0] and a[0] <= b[1])

day4(data4, has_overlap)

891

# Day 5

In [108]:
def make_move(positions, count, start, end):
  if count == 0:
    return
  item = positions[start].popleft()
  positions[end].appendleft(item)
  return make_move(positions, count-1, start, end)

def read_top(positions):
  return ''.join([positions[i][0] for i in range(len(positions))])

def day5(make_move):
  def parse5():
    with open('./data/day5.txt') as f:
      lines = f.readlines()
    positions = defaultdict(lambda: deque())
    line_length = len(lines[0])
    for n, line in enumerate(lines):
      for i in range(line_length//4):
        item = line[i*4:i*4+4].strip()
        if item == '1':
          return positions, lines[n+2:]
        if item:
          positions[i].append(re.match('\[([A-Z])\]', item)[1])
  positions, moves = parse5()
  for turn in moves:
    result = re.match('move (\d+) from (\d) to (\d)', turn)
    make_move(positions, int(result[1]), int(result[2])-1, int(result[3])-1)
  return read_top(positions)

print(day5(make_move))

RFFFWBPNS


In [109]:
def make_move_9001(positions, count, start, end):
  stack = deque()
  for _ in range(count):
    stack.appendleft(positions[start].popleft())
  positions[end].extendleft(stack)

print(day5(make_move_9001))

CQQBBJFCS


# Day 6

In [12]:
data6 = data(6)[0]

In [19]:
def day6(n):
  for i in range(len(data6)-n+1):
    if len(set(data6[i:i+n])) == n:
      return i + n

day6(4)

1262

In [15]:
day6(14)

3444

# Day 7

In [6]:
class Dir:
  def __init__(self, parent=None):
    self.parent = parent
    self.files = 0
    self.children = {}

def ls(item, cwd):
  if item.startswith('dir'):
    cwd.children[item[4:]] = Dir(cwd)
  else:
    cwd.files += int(item.split(' ')[0]) 

def cd(cmd, cwd, root):
  if cmd.startswith('cd'):
    cwd = root if cmd == 'cd /' else cwd.parent if '..' in cmd else cwd.children[cmd.split(' ')[1]]
  return cwd

def parse7(data):
  root = Dir()
  cwd = root
  for line in data:
    if line[0] == '$':
      cwd = cd(line[2:], cwd, root)
    else:
      ls(line, cwd)
  return root

data7 = []
def calculate_total_sizes(root):
  total = root.files
  for child in root.children:
    total += calculate_total_sizes(root.children[child])
  data7.append(total)
  return total
total = calculate_total_sizes(parse7(data(7)))

In [7]:
sum([x for x in data7 if x <= 100000])

1206825

In [8]:
target = total - (70000000 - 30000000)
next(filter(lambda x: x >= target, sorted(data7)))

9608311

# Day 8

In [289]:
data8 = np.array(data(8, lambda x: [int(n) for n in x]))
y, x = data8.shape

In [290]:
maskn, masks = [np.full([1, x], True)[0]], []
for i in range(x-1):
  forward = np.amax(data8[:i+1], 0)
  maskn.append(data8[i+1] > forward)
  backward = np.amax(data8[i+1:], 0)
  masks.append(data8[i] > backward)
masks.append(np.full([1, x], True)[0])

maskw, maske =  [np.full([y, 1], True)[:, 0]], []
for i in range(y-1):
  forward = np.amax(data8[:, :i+1], 1)
  maskw.append(data8[:, i+1] > forward)
  backward = np.amax(data8[:, i+1:], 1)
  maske.append(data8[:, i] > backward)
maske.append(np.full([y, 1], True)[:, 0])

mask = np.vstack(maskn) | np.vstack(masks) | np.vstack(maskw).transpose() | np.vstack(maske).transpose()
len(data8[np.where(mask)])

1816

In [288]:
def directional_score(geni, genj, axis=0):
  score = [np.zeros([1, x])[0]]
  for i in geni:
    current = data8[i] if axis == 0 else data8[:, i]
    visible = np.full([1, x], True)[0]
    result = np.zeros([1, x])[0]
    for j in genj(i):
      candidate = data8[j] if axis == 0 else data8[:, j]
      result += np.ones([1, x])[0] * visible
      visible = (current > candidate) & visible
    score.append(result)
  return np.vstack(score)

north = directional_score(range(1, x), lambda i: range(i-1, -1, -1))
south = np.flip(directional_score(range(x-2, -1, -1), lambda i: range(i+1, x)), 0)
west = directional_score(range(1, x), lambda i: range(i-1, -1, -1), 1).transpose()
east = np.flip(directional_score(range(x-2, -1, -1), lambda i: range(i+1, x), 1).transpose(), 1)
int(np.max(north * south * east * west))

383520

# Day 9

In [3]:
data9 = data(9, lambda x: x.split(' '))

In [6]:
def day9(data, knots=2):
  visited = set()
  state = [(0,0)] * knots

  def move_head(x, y, direction):
    match direction:
      case 'R':
        x += 1
      case 'L':
        x -= 1
      case 'U':
        y += 1
      case 'D':
        y -= 1
    return (x, y)

  def move_tail(headx, heady, tailx, taily):
    if abs(headx - tailx) > 1 and abs(heady - taily) > 1:
      return (headx + (1 if tailx > headx else -1), heady + (1 if taily > heady else -1))
    if abs(headx - tailx) > 1:
      return (headx + (1 if tailx > headx else -1), heady)
    if abs(heady - taily) > 1:
      return (headx, heady + (1 if taily > heady else -1))
    return (tailx, taily)

  def make_move(current, direction, count):
    for _ in range(count):
      current[0] = move_head(*current[0], direction)
      for i in range(1, len(current)):
        current[i] = move_tail(*current[i-1], *current[i])
      visited.add(current[-1])
    return current

  for move in data:
    state = make_move(state, move[0], int(move[1]))

  return len(visited)

day9(data9), day9(data9, 10)

(6181, 2386)

# Day 10

In [10]:
def parse10(data):
  q, i, state, op = deque([0]), 0, 1, 0

  def tock(i, state, op):
    if not op:
      if q:
        op = q.popleft()
    else:
      state += op
      op = 0
    i += 1
    return i, state, op

  for command in data:
    q.append(0 if len(command) == 1 else int(command[1]))
    i, state, op = tock(i, state, op)
    yield state
  while q:
    i, state, op = tock(i, state, op)
    yield state
  i, state, op = tock(i, state, op)
  yield state

data10 = lambda: parse10((data(10, lambda x: x.split(' '))))

In [11]:
def day10_1(data):
  signal = 0
  targets = [20, 60, 100, 140, 180, 220]
  for i, v in enumerate(data):
    if i+1 in targets:
      signal += (i+1) * v
  return signal
day10_1(data10())

14420

In [23]:
def day10_2(data):
  image = ''
  for i, v in enumerate(data):
    position, sprite = i+1, v
    if position%40 in range(sprite, sprite+3):
      image += '#'
    else:
      image += ' '
    if not position%40:
      image += '\n'
  return image
print(day10_2(data10()))

###   ##  #    ###  ###  ####  ##  #  # 
#  # #  # #    #  # #  #    # #  # #  ##
#  # #    #    #  # ###    #  #  # #  # 
###  # ## #    ###  #  #  #   #### #  ##
# #  #  # #    # #  #  # #    #  # #  ##
#  #  ### #### #  # ###  #### #  #  ##  
  


# Day 11

In [46]:
a = 'old * old'
x = lambda old: eval(a)

In [59]:
2.add

SyntaxError: invalid decimal literal (369896908.py, line 1)

In [115]:
data11 = []
ops = [ # Hardcoded
  lambda x: x*19,
  lambda x: x+6,
  lambda x: x*x,
  lambda x: x+3
]
# ops = [ # Hardcoded
#   lambda x: x*13,
#   lambda x: x+2,
#   lambda x: x+6,
#   lambda x: x*x,
#   lambda x: x+3,
#   lambda x: x*7,
#   lambda x: x+4,
#   lambda x: x+7
# ]

class Monkey:
  def __init__(self, items, op, n, a, b):
    self.items = items
    self.operation = op
    self.n = n
    self.a = a
    self.b = b
    self.count = 0

  def run(self, flag=False):
    t, f = data11[self.a], data11[self.b]
    for old in self.items:
      new = self.operation(old) // (1 if flag else 3)
      (f if new%self.n else t).items.append(new)
      self.count += 1
    self.items = []

  def __repr__(self):
    return f'{self.count}, {self.items}'

def parse11(data):
  monkey, monkeys = {}, []
  for line in data:
    if not line:
      monkeys.append(Monkey(**monkey))
    if m := re.match('Starting items: ([\d+, ]+)', line):
      monkey['items'] = [int(x) for x in m[1].split(', ')]
    elif m:= re.match('Operation: new = old (.) (.+)', line):
      monkey['op'] = ops[len(monkeys)]
    elif m:= re.match('Test: divisible by (\d+)', line):
      monkey['n'] = int(m[1])
    elif m:= re.match('If true: throw to monkey (\d)', line):
      monkey['a'] = int(m[1])
    elif m:= re.match('If false: throw to monkey (\d)', line):
      monkey['b'] = int(m[1])
  monkeys.append(Monkey(**monkey))
  return monkeys

def calculate_activity(monkeys):
  activity = sorted(monkeys, key=lambda x: x.count, reverse=True)
  # print(activity)
  return activity[0].count * activity[1].count

In [117]:
data11 = parse11(data(11))
for i in range(20):
  for monkey in data11:
    monkey.run()
calculate_activity(data11)

10605

In [116]:
data11 = parse11(data(11))
for i in range(100):
  for monkey in data11:
    monkey.run()
  print(calculate_activity(data11))

20
100
225
400
624
899
1224
1596
2021
2496
3024
3717
4224
5180
5621
6885
7221
8645
9405
10605
11766
12650
14036
15000
16758
17810
19448
21300
22484
24633
25740
28036
29392
31486
33286
35512
37422
39382
41800
43870
46420
49042
51736
53572
56386
58302
62230
63736
67840
69412
73150
75330
79222
82636
84952
89082
90882
95770
97636
102060
105280
108550
113212
115920
121432
122830
128502
130662
136510
140230
144000
148590
151690
157192
161986
166852
170962
175122
179332
184450
188770
194456
198000
204279
207911
214344
218999
223704
229416
233264
241056
243522
250480
254500
260590
266752
271942
280350
282472
291040
